In [1]:
from numpy.random import seed
import pandas as pd
import numpy as np
from sklearn.preprocessing import minmax_scale
# from keras_diagram import ascii
from keras.layers.convolutional import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
import keras

data = pd.read_csv("creditcard.csv")

# Normalise and reshape the Amount column, so it's values lie between -1 and 1
from sklearn.preprocessing import StandardScaler
data['norm_Amount'] = StandardScaler().fit_transform(data['Amount'].reshape(-1,1))

# Drop the old Amount column and also the Time column as we don't want to include this at this stage
data = data.drop(['Amount'], axis=1)

data.sort_values(['Time'])


Using TensorFlow backend.
/Users/HarryG1/Documents/University/Year_3/DeepLearning-CreditCardFraud/devEnv/lib/python2.7/site-packages/ipykernel_launcher.py:18: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Class,norm_Amount
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0,0.244964
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0,-0.342475
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0,1.160686
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0,0.140534
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0,-0.073403
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,0,-0.338556
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,0,-0.333279
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,0,-0.190107
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,0,0.019392
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,0,-0.338516


In [2]:
# Assign variables x and y corresponding to row data and it's class value
X = data.ix[:, data.columns != 'Class']
y = data.ix[:, data.columns == 'Class']

def generate_train_test_sample(x_data, y_data): 
    ''' 1) Generate new, random train-test split
        2) Random smote oversample the train data, keeping test data unseen
        3) Use this new train-test split to fit and test model
    '''

    X_train, X_test, y_train, y_test = train_test_split(x_data,y_data,test_size = 0.3)

    from collections import Counter
    from imblearn.over_sampling import SMOTE
    sm = SMOTE()
    X_res, y_res = sm.fit_sample(X_train, y_train)
    print('Resampling the data with SMOTE. . .')
    print('Resampled training dataset shape {}'.format(Counter(y_res)))

    return X_res, y_res, X_test, y_test

def generate_train_test_timeseries(data,cuttoff_percent=0.7):
    cutoff = math.floor(X.shape[0] *cuttoff_percent) 
    
    X_train = data.ix[0:cutoff, data.columns != 'Class']
    y_train = data.ix[0:cutoff, data.columns == 'Class']
    X_test = data.ix[cutoff:, data.columns != 'Class']
    y_test = data.ix[cutoff:, data.columns == 'Class']
    
    return X_train, y_train, X_test, y_test
    
########################################################################

# X_res, y_res, X_test, y_test = generate_train_test_sample(X, y)

# print X_res.shape, type(X_res)
# print y_res.shape

# X_train = X_res.reshape(X_res.shape[0], 29, 1)
# Y_train = y_res.reshape(y_res.shape[0], 1)
# X_test = X_test.values.reshape(X_test.values.shape[0], 29, 1)
# Y_test = y_test.values.reshape(y_test.values.shape[0], 1)

# Y_test = keras.utils.to_categorical(Y_test)
# Y_train = keras.utils.to_categorical(Y_train)
# print Y_test.shape
# print Y_train.shape




/Users/HarryG1/Documents/University/Year_3/DeepLearning-CreditCardFraud/devEnv/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [3]:
data[data['Class']==1]

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Class,norm_Amount
541,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,1,-0.353229
623,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,1,1.761758
4920,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,1,0.606031
6108,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,1,-0.117342
6329,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1,-0.349231
6331,7526.0,0.008430,4.137837,-6.240697,6.675732,0.768307,-3.353060,-1.631735,0.154612,-2.795892,...,0.364514,-0.608057,-0.539528,0.128940,1.488481,0.507963,0.735822,0.513574,1,-0.349231
6334,7535.0,0.026779,4.132464,-6.560600,6.348557,1.329666,-2.513479,-1.689102,0.303253,-3.139409,...,0.370509,-0.576752,-0.669605,-0.759908,1.605056,0.540675,0.737040,0.496699,1,-0.349231
6336,7543.0,0.329594,3.712889,-5.775935,6.078266,1.667359,-2.420168,-0.812891,0.133080,-2.214311,...,0.156617,-0.652450,-0.551572,-0.716522,1.415717,0.555265,0.530507,0.404474,1,-0.349231
6338,7551.0,0.316459,3.809076,-5.615159,6.047445,1.554026,-2.651353,-0.746579,0.055586,-2.678679,...,0.208828,-0.511747,-0.583813,-0.219845,1.474753,0.491192,0.518868,0.402528,1,-0.349231
6427,7610.0,0.725646,2.300894,-5.329976,4.007683,-1.730411,-1.732193,-3.968593,1.063728,-0.486097,...,0.589669,0.109541,0.601045,-0.364700,-1.843078,0.351909,0.594550,0.099372,1,-0.349231


In [4]:
import math


X_train, y_train, X_test, y_test = generate_train_test_timeseries(data)
print X_train.shape
print y_train.shape



(199365, 30)
(199365, 1)


In [5]:
X_test.head(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,norm_Amount
199364,132929.0,-1.504565,-1.047164,2.658591,-0.246340,0.342667,0.182682,-0.424257,0.300896,0.638538,...,0.560674,0.371965,0.798937,-0.048231,-0.437821,0.412388,-0.085327,-0.131134,-0.127378,0.120864
199365,132929.0,2.152590,-0.900447,-1.092456,-0.994969,-0.613447,-0.760012,-0.530218,-0.222248,-0.214416,...,0.060715,-0.281725,-0.987236,0.358646,-0.782338,-0.480245,-0.647930,-0.023563,-0.044354,-0.129536
199366,132929.0,1.811278,0.224393,0.542888,3.606222,-0.162910,1.171241,-0.874598,0.419449,-0.546470,...,-0.198616,0.109290,0.345703,0.365984,0.663253,-0.531272,-0.225013,0.032563,-0.022693,-0.350071
199367,132929.0,-1.524832,-0.544095,-0.288824,-0.916791,3.524219,-1.830070,1.375925,-0.814647,-0.648070,...,-0.209666,0.272832,1.245119,-0.867245,0.838885,0.860487,-0.081074,-0.252743,-0.377957,-0.229689
199368,132930.0,0.099135,0.873164,-0.362487,-0.612234,0.747229,-0.629384,0.755890,0.126794,-0.051667,...,-0.090654,-0.302759,-0.840999,0.115856,0.613080,-0.528392,0.092546,0.207960,0.071553,-0.346113
199369,132930.0,2.005556,-0.146704,-1.019430,0.330239,-0.180449,-0.953319,0.053860,-0.165823,0.352282,...,-0.241755,-0.235137,-0.613122,0.342269,0.063677,-0.359509,0.174021,-0.078440,-0.069642,-0.331760
199370,132930.0,-0.972978,0.359825,0.979863,-0.864804,1.876989,-0.809730,1.125462,-0.403589,-0.531036,...,0.298142,0.245493,0.713429,-0.554851,0.543635,1.176300,-0.036967,-0.183880,-0.160094,-0.280424
199371,132931.0,-0.040660,1.299237,-1.340674,0.651780,1.200997,-0.191688,1.097722,0.244725,-1.303420,...,-0.144117,0.385317,0.984711,0.004701,0.218244,-0.767317,-0.550411,0.141207,0.204165,-0.201702
199372,132931.0,1.092328,-1.860651,-1.054998,1.470728,0.051875,2.633389,-0.620770,0.717955,1.483445,...,0.368666,0.539129,1.205393,-0.344476,-0.902377,-0.009404,-0.244045,0.038282,0.001911,1.158047
199373,132931.0,-1.325669,0.625022,0.215274,0.039639,2.302044,4.676421,-0.046687,1.290433,-0.109319,...,-0.074782,-0.529958,-1.405320,-0.245663,0.652001,0.311430,-0.831220,-0.323458,-0.310960,-0.069525


In [6]:
print X_train.shape
print y_train.shape
print X_test.shape
print y_test.shape

(199365, 30)
(199365, 1)
(85443, 30)
(85443, 1)


In [7]:
def reshape_to_batches(a, batch_size):
    # pad with zeros if the length is not divisible by the batch_size
    batch_num = np.ceil((float)(a.shape[0]) / batch_size)
    modulo = batch_num * batch_size - a.shape[0]
    if modulo != 0:
        pad = np.zeros((int(modulo), a.shape[1]))
        a = np.vstack((a, pad))
    return np.array(np.split(a, batch_num))

In [8]:
X_train_batch = reshape_to_batches(X_train, 100)

In [9]:
X_train_batch.shape

(1994, 100, 30)

In [10]:
y_test.shape

(85443, 1)

In [11]:
y_test = keras.utils.to_categorical(y_test)
y_train = keras.utils.to_categorical(y_train)

In [136]:
y_test.shape

(85443, 2)

In [12]:
y_train_batch = reshape_to_batches(y_train, 100)
y_test_batch = reshape_to_batches(y_test, 100)
X_test_batch = reshape_to_batches(X_test, 100)

In [13]:
print X_train_batch.shape
print X_test_batch.shape
print y_train_batch.shape
print y_test_batch.shape

(1994, 100, 30)
(855, 100, 30)
(1994, 100, 2)
(855, 100, 2)


In [23]:
#import all dependencies
from keras.layers import Input, Dense, Conv1D
from keras.models import Model

In [24]:
inputs = Input(shape=(100, 30)) # This returns a tensor

In [25]:
# a layer instance is callable on a tensor, and returns a tensor
conv1 = Conv1D(32, (5), # 32 filters with a window of width 5
strides=1, # think autoregression
padding='causal', # forward in time
dilation_rate=1, # ignore this and everything that follows are default parameters
activation='relu',
use_bias=True,
kernel_initializer='glorot_uniform',
bias_initializer='zeros',
kernel_regularizer=None, # no regularisation for the moment
bias_regularizer=None,
activity_regularizer=None,
kernel_constraint=None,
bias_constraint=None)(inputs) # syntax to chain layers: Layer(...)(PreviousLayer)

In [26]:
fc1 = Dense(64, activation='relu')(conv1)

In [27]:
predictions = Dense(2, activation='softmax')(fc1)

In [28]:
model = Model(inputs=inputs,
outputs=predictions)
model.compile(optimizer='rmsprop',
loss='categorical_crossentropy',metrics=['accuracy'])

In [29]:
model.fit(X_train_batch, y_train_batch, epochs=30)

Epoch 1/30
1994/1994 [==============================] - 1s 478us/step - loss: 2.1534 - acc: 0.8664
Epoch 2/30
1994/1994 [==============================] - 1s 335us/step - loss: 0.0311 - acc: 0.9981
Epoch 3/30
1994/1994 [==============================] - 1s 336us/step - loss: 0.0311 - acc: 0.9981
Epoch 4/30
1994/1994 [==============================] - 1s 340us/step - loss: 0.0311 - acc: 0.9981
Epoch 5/30
1994/1994 [==============================] - 1s 341us/step - loss: 0.0310 - acc: 0.9981
Epoch 6/30
1994/1994 [==============================] - 1s 344us/step - loss: 0.0310 - acc: 0.9981
Epoch 7/30
1994/1994 [==============================] - 1s 340us/step - loss: 0.0310 - acc: 0.9981
Epoch 8/30
1994/1994 [==============================] - 1s 369us/step - loss: 0.0310 - acc: 0.9981
Epoch 9/30
1994/1994 [==============================] - 1s 367us/step - loss: 0.0310 - acc: 0.9981
Epoch 10/30
1994/1994 [==============================] - 1s 345us/step - loss: 0.0310 - acc: 0.9981
Epoch 11/

In [30]:
y_pred = model.predict(X_test_batch)

In [31]:
y_pred_3d = y_pred

# a function to reshape batches into the original shape
def _3d_to_2d(arr):
    return arr.reshape(arr.shape[0] * arr.shape[1], arr.shape[2])

In [32]:
from sklearn.metrics import confusion_matrix
cutt_off_tr = 0.5
y_pred[np.where(y_pred>=cutt_off_tr)] = 1
y_pred[np.where(y_pred<cutt_off_tr)]  = 0

y_pred_2d = _3d_to_2d(y_pred)[:,1]
y_test_2d = _3d_to_2d(y_test_batch)[:,1]

print(confusion_matrix(
_3d_to_2d(y_test_batch)[:, 1],
_3d_to_2d(y_pred)[:, 1]))

prfs = precision_recall_fscore_support(y_test_2d, y_pred_2d, labels=[1])
prfs0 = precision_recall_fscore_support(y_test_2d, y_pred_2d, labels=[0])
print prfs
print "\n"
print prfs0

[[85392     0]
 [  108     0]]
(array([ 0.]), array([ 0.]), array([ 0.]), array([108]))


(array([ 0.99873684]), array([ 1.]), array([ 0.99936802]), array([85392]))


In [14]:
conv1d = Conv1D(32, (5), input_shape=(100, 30), strides=1,padding='causal')

In [15]:
seed(2017)
conv = Sequential()
conv.add(conv1d)
conv.add(Dense(64, activation='relu'))
conv.add(Dense(2, activation='softmax'))

In [17]:
sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
conv.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
conv.fit(X_train_batch, y_train_batch, epochs = 30, verbose = 50)

Epoch 1/30
Epoch 2/30
Epoch 3/30
Epoch 4/30
Epoch 5/30
Epoch 6/30
Epoch 7/30
Epoch 8/30
Epoch 9/30
Epoch 10/30
Epoch 11/30
Epoch 12/30
Epoch 13/30
Epoch 14/30
Epoch 15/30
Epoch 16/30
Epoch 17/30
Epoch 18/30
Epoch 19/30
Epoch 20/30
Epoch 21/30
Epoch 22/30
Epoch 23/30
Epoch 24/30
Epoch 25/30
Epoch 26/30
Epoch 27/30
Epoch 28/30
Epoch 29/30
Epoch 30/30


In [18]:
y_pred = conv.predict(X_test_batch)

In [19]:
y_pred_3d = y_pred

# a function to reshape batches into the original shape
def _3d_to_2d(arr):
    return arr.reshape(arr.shape[0] * arr.shape[1], arr.shape[2])



In [20]:
print y_pred_2d.shape
print y_test_2d.shape

NameError: name 'y_pred_2d' is not defined

In [21]:
from sklearn.metrics import confusion_matrix
cutt_off_tr = 0.5
y_pred[np.where(y_pred>=cutt_off_tr)] = 1
y_pred[np.where(y_pred<cutt_off_tr)]  = 0

y_pred_2d = _3d_to_2d(y_pred)[:,1]
y_test_2d = _3d_to_2d(y_test_batch)[:,1]

print(confusion_matrix(
_3d_to_2d(y_test_batch)[:, 1],
_3d_to_2d(y_pred)[:, 1]))

prfs = precision_recall_fscore_support(y_test_2d, y_pred_2d, labels=[1])
prfs0 = precision_recall_fscore_support(y_test_2d, y_pred_2d, labels=[0])
print prfs
print "\n"
print prfs0

[[85392     0]
 [  108     0]]
(array([ 0.]), array([ 0.]), array([ 0.]), array([108]))


(array([ 0.99873684]), array([ 1.]), array([ 0.99936802]), array([85392]))


/Users/HarryG1/Documents/University/Year_3/DeepLearning-CreditCardFraud/devEnv/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
y_pred_2d

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

print(roc_auc_score(y_test_2d[:, 1],
y_pred_2d[:, 1]))

In [22]:
from collections import Counter
Counter(y_pred_2d)


Counter({0.0: 85500})